# Setup

In [ ]:
%pip install -r requirements.txt

I0000 00:00:1740269245.243385 2390576 fork_posix.cc:75] Other threads are currently calling into gRPC, skipping fork() handlers



[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [32]:
import requests
import os
import re
import json
import time
import pyht
from openai import OpenAI
from dotenv import load_dotenv
import phoenix as px
from phoenix.otel import register
from openinference.instrumentation.openai import OpenAIInstrumentor

# INPUT = BOOK TITLE
## What's Your Favorite Novel (ideally never been made a movie)?

In [15]:
book_title = "Klara and the Sun"

In [16]:
# Launch Phoenix
px.launch_app()

# defaults to endpoint="http://localhost:4317"
tracer_provider = register(
  project_name="fourth_wall_app", # Default is 'default'
  endpoint="http://localhost:4317",  # Sends traces using gRPC
) 

OpenAIInstrumentor().instrument(tracer_provider=tracer_provider)

Existing running Phoenix instance detected! Shutting it down and starting a new instance...
Overriding of current TracerProvider is not allowed
Attempting to instrument while already instrumented


🌍 To view the Phoenix app in your browser, visit http://localhost:6006/
📖 For more information on how to use Phoenix, check out https://docs.arize.com/phoenix
🔭 OpenTelemetry Tracing Details 🔭
|  Phoenix Project: fourth_wall_app
|  Span Processor: SimpleSpanProcessor
|  Collector Endpoint: localhost:4317
|  Transport: gRPC
|  Transport Headers: {'user-agent': '****'}
|  
|  Using a default SpanProcessor. `add_span_processor` will overwrite this default.
|  
|  `register` has set this TracerProvider as the global OpenTelemetry default.
|  To disable this behavior, call `register` with `set_global_tracer_provider=False`.



In [17]:
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")
client = OpenAI(
    api_key=openai_api_key
)

In [18]:
def get_wikipedia_summary(title):
    """
    Fetches the first extract of a Wikipedia article using the public MediaWiki API.
    Returns a text summary or an empty string if not found.
    """
    base_url = "https://en.wikipedia.org/w/api.php"
    params = {
        "action": "query",
        "prop": "extracts",
        "explaintext": True,
        "format": "json",
        "titles": title
    }
    try:
        response = requests.get(base_url, params=params, timeout=10)
        response.raise_for_status()
        data = response.json()
        pages = data.get("query", {}).get("pages", {})
        for page_id, page_content in pages.items():
            if "extract" in page_content:
                # This is a raw textual extract from Wikipedia
                return page_content["extract"]
    except requests.RequestException as e:
        print(f"[Wikipedia] Error fetching summary: {e}")

    return ""

In [19]:
summary = get_wikipedia_summary(book_title)
print(summary)

Klara and the Sun is the eighth novel by the British writer Kazuo Ishiguro, published on 2 March 2021. It is a dystopian science fiction story.
Set in the U.S. in an unspecified future, the book is told from the point of view of Klara, a solar-powered AF (Artificial Friend), who is chosen by Josie, a sickly child, to be her companion. 
The novel was longlisted for the 2021 Booker Prize.


== Plot ==
The novel is set in a dystopian future in which some children are genetically engineered ("lifted") for enhanced academic ability. As schooling is provided entirely at home by on-screen tutors, opportunities for socialization are limited and parents who can afford it often buy their children androids as companions. The book is narrated by one such Artificial Friend (AF) called Klara. Although Klara is exceptionally intelligent and observant, her knowledge of the world is limited.
From the window of the store in which she is for sale, Klara learns about the world outside and watches the Sun,

# Get a High Level Trailer Script for your book

In [20]:
def query_openai(system_prompt, user_prompt):
    """Send a prompt to OpenAI and return the response."""
    
    completion = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ]
    )
    
    return completion.choices[0].message.content

In [24]:
# Load the system prompt from system_prompts/story_analysis_prompt.md
with open("system_prompts/story_analysis_prompt.md", "r") as file:
    system_prompt = file.read()

# Generate a response using the OpenAI API
trailer_description = query_openai(system_prompt, summary)

In [29]:
# Step 1: Extract the JSON part from between the triple backticks
pattern = r"```json\s*(.*?)```"
match = re.search(pattern, trailer_description, flags=re.DOTALL)

if match:
    # Step 2: Parse the extracted string as JSON
    json_str = match.group(1)
    data = json.loads(json_str)

    # Now data is a Python dictionary you can work with.
    print(data["clips"][0]["visual"])
    # -> "A serene view through the dusty window of a store..."
else:
    print("No JSON code block found!")

A serene view through the dusty window of a store, showcasing Klara, an elegant yet expressionless android, observing the bustling world outside.


In [31]:
trailer_clips = data["clips"]
print(trailer_clips)

[{'visual': 'A serene view through the dusty window of a store, showcasing Klara, an elegant yet expressionless android, observing the bustling world outside.', 'timing': '0:00-0:10', 'audio': 'A soft hum of city ambiance punctuated by the distant laughter of children and the occasional honk of a car horn.', 'narration': 'In a world where children are engineered and companionship is bought, one Artificial Friend observes life beyond her glass cocoon.'}, {'visual': 'The bright Sun casts long shadows as Klara stands immobile, her eyes reflecting its golden rays with a hint of reverence.', 'timing': '0:11-0:20', 'audio': "Wind softly swirls, leaves rustling as a gentle melody echoes the Sun's warmth.", 'narration': 'To Klara, the Sun is a benevolent entity – a giver of life, a symbol of hope.'}, {'visual': "Josie, frail yet spirited, grips her mother's hand as they step into the store, locking eyes with Klara.", 'timing': '0:21-0:30', 'audio': 'Tender piano notes interwoven with the light

# Create Video and Audio

In [33]:
from lumaai import AsyncLumaAI, LumaAI
from dotenv import load_dotenv

import requests
import time
import os

from pyht import Client
from pyht.client import TTSOptions

load_dotenv()
client = LumaAI()

In [ ]:
def text_to_speech(narrative_text, clip_number, book_title):
    client = Client(
        user_id=os.getenv("PLAY_HT_USER_ID"),
        api_key=os.getenv("PLAY_HT_API_KEY"),
    )
    options = TTSOptions(voice="s3://voice-cloning-zero-shot/775ae416-49bb-4fb6-bd45-740f205d20a1/jennifersaad/manifest.json")
    # Open a file to save the audio
    with open(f"first_clips/{book_title}/{clip_number}/narration.wav", "wb") as audio_file:
        for chunk in client.tts(narrative_text, options, voice_engine = 'PlayDialog-http'):
            # Write the audio chunk to the file
            audio_file.write(chunk)

[{'visual': 'A serene view through the dusty window of a store, showcasing Klara, an elegant yet expressionless android, observing the bustling world outside.',
  'timing': '0:00-0:10',
  'audio': 'A soft hum of city ambiance punctuated by the distant laughter of children and the occasional honk of a car horn.',
  'narration': 'In a world where children are engineered and companionship is bought, one Artificial Friend observes life beyond her glass cocoon.'},
 {'visual': 'The bright Sun casts long shadows as Klara stands immobile, her eyes reflecting its golden rays with a hint of reverence.',
  'timing': '0:11-0:20',
  'audio': "Wind softly swirls, leaves rustling as a gentle melody echoes the Sun's warmth.",
  'narration': 'To Klara, the Sun is a benevolent entity – a giver of life, a symbol of hope.'},
 {'visual': "Josie, frail yet spirited, grips her mother's hand as they step into the store, locking eyes with Klara.",
  'timing': '0:21-0:30',
  'audio': 'Tender piano notes interwo

In [35]:
# create a directory in first_clips withthe book title and a subdirectory for each clip
try:
      os.makedirs(f'first_clips/{book_title}')
except FileExistsError:
      pass


In [38]:
def text_to_speech(narrative_text, clip_number, book_title):
    client = Client(
        user_id=os.getenv("PLAY_HT_USER_ID"),
        api_key=os.getenv("PLAY_HT_API_KEY"),
    )
    options = TTSOptions(voice="s3://voice-cloning-zero-shot/775ae416-49bb-4fb6-bd45-740f205d20a1/jennifersaad/manifest.json")
    # Open a file to save the audio
    with open(f"first_clips/{book_title}/{clip_number}/narration.wav", "wb") as audio_file:
        for chunk in client.tts(narrative_text, options, voice_engine = 'PlayDialog-http'):
            # Write the audio chunk to the file
            audio_file.write(chunk)

In [39]:
# generate
for i, clip in enumerate(trailer_clips):
   generation = client.generations.create(
   prompt=clip['visual'],
   model='ray-2',
   resolution='540p',
   duration='5s',
   )

   completed = False
   while not completed:
      generation = client.generations.get(id=generation.id)
      if generation.state == "completed":
         completed = True
      elif generation.state == "failed":
         raise RuntimeError(f"Generation failed: {generation.failure_reason}")
      print("...")
      time.sleep(3)
   

   video_url = generation.assets.video
   response = requests.get(video_url, stream=True)
   # check to see if the directory exists
   video_path = f'first_clips/{book_title}/{i}'
   try:
      os.makedirs(video_path)
   except FileExistsError:
      pass
   with open(f'first_clips/{book_title}/{i}/{generation.id}.mp4', 'wb') as file:
      file.write(response.content)
   print(f"Video Clip downloaded as {generation.id}.mp4")
   
   text_to_speech(clip['narration'], i, book_title)
   print(f"Narration for clip {i} generated")


...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
Video Clip downloaded as f4a7c2ce-e686-4594-9c52-4ef90d49edf6.mp4


Narration for clip 0 generated
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
Video Clip downloaded as 41ecc800-de58-4d05-86cf-bd9fadb603cd.mp4


Narration for clip 1 generated
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
Video Clip downloaded as 51b2c707-8479-4469-9ff0-879114a51156.mp4


Narration for clip 2 generated
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
Video Clip downloaded as e402ac78-cae4-4bcb-beb8-eeabebf19aea.mp4


Narration for clip 3 generated
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
Video Clip downloaded as a927fa42-c70a-4d03-b348-5bd8607fe100.mp4


Narration for clip 4 generated
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
Video Clip downloaded as c13b8e5e-d633-45dd-86ab-91ec2ba90009.mp4


Narration for clip 5 generated


BadRequestError: Error code: 400 - {'detail': 'Insufficient credits'}

# Stitch Together Audio and Video

In [ ]:
from moviepy import VideoFileClip, AudioFileClip, concatenate_videoclips

def stitch_trailer(book_title, number_of_clips):

    clips = []

    for i in range(number_of_clips):
        folder_path = f"{book_title}/{i+1}/"

        video_clip = VideoFileClip(f"{folder_path}video.mp4")
        audio_clip = AudioFileClip(f"{folder_path}/audio.wav")

        audio_clip = audio_clip.set_duration(video_clip.duration)

        # Combine video with audio
        final_clip = video_clip.set_audio(audio_clip)

        # Option 1: Write the final video to a file
        final_clip.write_videofile(f"{folder_path}/{i+1}.mp4", codec="libx264", audio_codec="aac")

        clips.append(VideoFileClip(f"{folder_path}/{i+1}.mp4"))

    final_clip = concatenate_videoclips(clips)

    final_clip.write_videofile(f"{book_title}/final_clip.mp4", codec="libx264", audio_codec="aac")